Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [24]:
let name = "Anumala Venu Madhava Reddy"
let rollno = "CS18B051"

val name : string = "Anumala Venu Madhava Reddy"


val rollno : string = "CS18B051"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

## A Prolog interpreter in OCaml

In this assignment, you will implement a Prolog interpreter in OCaml with support for backtracking (recover from bad choices) and choice points (produce multiple results).

We will start with the definition of terms, predicates, clauses, goals and programs.

In [25]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

type constant = Constant of string


type variable = Variable of string


type func = Function of string * term list
and term = C of constant | V of variable | F of func


type predicate = Predicate of string * term list


type head = predicate


type body = predicate list


type clause = Fact of head | Rule of head * body


type goal = predicate list


type program = clause list


The following stringification functions should help you debug the interpreter. 

In [26]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

val string_of_f_list : ('a -> string) -> 'a list -> string = <fun>


val string_of_term : term -> string = <fun>


val string_of_predicate : predicate -> string = <fun>


val string_of_predicate_list : predicate list -> string = <fun>


val string_of_goal : predicate list -> string = <fun>


val string_of_clause : clause -> string = <fun>


val string_of_program : clause list -> string = <fun>


Below are more helper functions:

In [27]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

val var : string -> variable = <fun>


val var_t : string -> term = <fun>


val const : string -> constant = <fun>


val const_t : string -> term = <fun>


val func : string -> term list -> func = <fun>


val func_t : string -> term list -> term = <fun>


val pred : string -> term list -> predicate = <fun>


val fact : head -> clause = <fun>


val rule : head -> body -> clause = <fun>


## Problem 1

In this problem, you will implement the function:

```ocaml
occurs_check : variable -> term -> bool
```

`occurs_check v t` returns `true` if the variable `v` occurs in `t`. 

In [28]:
let rec fun_help f v tl acc = 
    match tl with
    |[] -> acc
    | h::t' -> if (f v h)=true then true else fun_help f v t' acc
    
let rec occurs_check v t = 
    match t with
    |C (Constant c) -> false
    |V (Variable v') ->  (v = Variable v')
    |F (Function (f,tl)) -> fun_help  (occurs_check) v tl false

val fun_help : ('a -> 'b -> bool) -> 'a -> 'b list -> bool -> bool = <fun>


val occurs_check : variable -> term -> bool = <fun>


In [29]:
(* 5 points *)
assert (occurs_check (var "X") (var_t "X"));
assert (not (occurs_check (var "X") (var_t "Y")));
assert (occurs_check (var "X") (func_t "f" [var_t "X"]))

- : unit = ()


## Problem 2

Implement the following functions which return the variables contained in them:

```ocaml
variables_of_term      : term -> VarSet.t
variables_of_predicate : predicate -> VarSet.t
variables_of_clause    : clause -> VarSet.t
```

In [30]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* API Docs for Set : https://caml.inria.fr/pub/docs/manual-ocaml/libref/Set.S.html *)

let rec help_fun f  acc' tl =
    match tl with 
    | [] -> acc'
    | h::tl1 -> help_fun f  (f h acc') tl1
    
let rec variables_of_term1 te acc = 
    match te with
    |C (Constant c) -> acc
    |V (Variable v') ->  VarSet.add (Variable v') acc
    |F (Function (f,tl)) -> help_fun (variables_of_term1) acc tl
    
let variables_of_term te = variables_of_term1 te (VarSet.empty) 
  
let  variables_of_predicate (Predicate (_, l)) = help_fun (variables_of_term1) (VarSet.empty) l
  
let variables_of_predicate1 (Predicate ( _ , l)) acc = help_fun (variables_of_term1) acc l

let variables_of_clause c = 
    match c with
    |Fact h -> variables_of_predicate h
    |Rule (h,b) -> help_fun (variables_of_predicate1) (variables_of_predicate h) b 

File "[30]", line 1, characters 64-82:
1 | module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                    ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module VarSet :
  sig
    type elt = variable
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val disjoint : t -> t -> bool
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    

val help_fun : ('a -> 'b -> 'b) -> 'b -> 'a list -> 'b = <fun>


val variables_of_term1 : term -> VarSet.t -> VarSet.t = <fun>


val variables_of_term : term -> VarSet.t = <fun>


val variables_of_predicate : predicate -> VarSet.t = <fun>


val variables_of_predicate1 : predicate -> VarSet.t -> VarSet.t = <fun>


val variables_of_clause : clause -> VarSet.t = <fun>


In [31]:
(* 5 points *)
assert (variables_of_term (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_predicate (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]])= 
        VarSet.of_list [var "X"; var "Y"])

- : unit = ()


## Problem 3

The value of type `term Substitution.t` is a OCaml map whose key is of type `variable` and value is of type `term`. In other words, it is map from variables to terms. Implement substitution functions that takes a `term Substitution.t` and uses that to perform the substitution:

```ocaml
substitute_in_term : term Substitution.t -> term -> term
substitute_in_predicate : term Substitution.t -> predicate -> predicate
substitute_in_clause : term Substitution.t -> clause -> clause
```

In [32]:
module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* See API docs for OCaml Map: https://caml.inria.fr/pub/docs/manual-ocaml/libref/Map.S.html *)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"

let rec fun3 f s l acc f' =
    match l with
    |[] -> F (Function (f',acc))
    |hd::tl' ->  (fun3 f s tl' (acc @ [(f s hd)]) f' )
    
let rec substitute_in_term s t =
    match t with
    |C (Constant c) -> C (Constant c)
    |V (Variable v) ->  
        (match (Substitution.find_opt (Variable v) s) with
        |Some x -> x
        |None -> V (Variable v)
        )
    |F (Function (f',tl)) -> fun3 substitute_in_term s tl [] f'

let rec fun32 f s l acc p =
    match l with
    |[] -> Predicate (p,acc)
    |hd::tl' ->  (fun32 f s tl' (acc @ [(f s hd)]) p )
    
let  substitute_in_predicate s (Predicate (p,l)) = fun32 substitute_in_term s l [] p
  
let rec fun33 f s l acc =
    match l with
    |[] -> acc
    |hd::tl' ->  (fun33 f s tl' (acc @ [(f s hd)])  )
    
let substitute_in_clause s c =
    match c with
    |Fact hd -> Fact (substitute_in_predicate s hd)
    |Rule (hd,bd) -> Rule ( (substitute_in_predicate s hd) , (fun33 substitute_in_predicate s bd []) )

File "[32]", line 1, characters 70-88:
1 | module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                          ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module Substitution :
  sig
    type key = variable
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a 

val string_of_substitution : term Substitution.t -> string = <fun>


val fun3 : ('a -> 'b -> term) -> 'a -> 'b list -> term list -> string -> term =
  <fun>


val substitute_in_term : term Substitution.t -> term -> term = <fun>


val fun32 :
  ('a -> 'b -> term) -> 'a -> 'b list -> term list -> string -> predicate =
  <fun>


val substitute_in_predicate : term Substitution.t -> predicate -> predicate =
  <fun>


val fun33 : ('a -> 'b -> 'c) -> 'a -> 'b list -> 'c list -> 'c list = <fun>


val substitute_in_clause : term Substitution.t -> clause -> clause = <fun>


In [33]:
(* 10 points *)
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty)
in
assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_clause s (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        (fact @@ pred "p" [var_t "Y"; const_t "0"; const_t "a"]));
assert (substitute_in_clause s (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]) = 
        (rule (pred "p" [var_t "Y"; const_t "0"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]))

- : unit = ()


We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [34]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

val counter : int ref = {contents = 0}


val fresh : unit -> term = <fun>


val freshen : clause -> clause = <fun>


For example, 

In [35]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

val c : clause =
  Rule
   (Predicate ("p", [V (Variable "X"); V (Variable "Y"); C (Constant "a")]),
   [Predicate ("q", [V (Variable "X"); C (Constant "b"); C (Constant "a")])])


- : string = "p(X, Y, a) :- q(X, b, a)."


- : string = "p(_G0, _G1, a) :- q(_G0, b, a)."


## Problem 4

Implement the function:

```ocaml
mgu : term -> term -> term Substitution.t
```

returns the most general unifier of the given terms. The function should raise the exception `Not_unfifiable`, if the given terms are not unifiable. 

Using `mgu`, implement the function:

```ocaml
mgu_predicate : predicate -> predicate -> term Substitution.t
```

which returns the most general unifier of the given predicates. 

In [36]:
exception Not_unifiable


let rec leng1 l acc = 
    match l with
    |[] -> acc
    |hd::tl -> leng1 tl (acc+1)
    
let leng l = leng1 l 0

let rec hold_fun f acc l1 l2 =
    match (l1,l2) with
    | ([],[]) -> acc
    |( (h1::t1), (h2::t2) ) -> hold_fun f (f h1 h2 acc) t1 t2

let composition o s = let u = ref o in
    Substitution.iter
    (
    let rec func a v =
        if Substitution.mem a o then ()
        else (
            (if Substitution.exists (fun b v' -> occurs_check a v') o then 
                Substitution.iter (
                    let func b v' = 
                        (if (occurs_check a v') then (u := Substitution.remove b !u;
                            if (not(v=V(b))) then 
                                u := Substitution.add b (substitute_in_term (Substitution.singleton a v) v') !u;)
                        )
                        in func
                ) 
                o;
            );
            u := Substitution.add a v !u;
        )
    in func
    ) s;!u;;

let rec mgu1 t1 t2 o = 
    let x = substitute_in_term o t1 in 
        let y = substitute_in_term o t2 in
            match (x,y) with
            |((V (Variable v1)), (V (Variable v2))) -> if (v1 = v2) then o 
                        else let s = Substitution.add (Variable v1) (y) (Substitution.empty) in composition o s
            |((V (Variable v1)),(C (Constant c1))) ->
                            let s = Substitution.add (Variable v1) (y) (Substitution.empty) in composition o s
            |((V (Variable v)),F (Function (f,tl)) ) -> if occurs_check (Variable v) y = false then 
                            let s = Substitution.add (Variable v) (y) (Substitution.empty) in composition o s
                        else raise(Not_unifiable)
            |((C (Constant c1)),(V (Variable v2))) ->
                            let s = Substitution.add (Variable v2) (x) (Substitution.empty) in composition o s
            |((C (Constant c1)),(C (Constant c2))) -> if c1=c2 then o else raise (Not_unifiable)
            |((C (Constant c)),(F (Function (f,tl)))) -> raise (Not_unifiable)
            |((F (Function (f,tl))),(V (Variable v))) -> if occurs_check (Variable v) x = false then
                            let s = Substitution.add (Variable v) (x) (Substitution.empty) in composition o s
                        else raise (Not_unifiable)
            |((F (Function (f,tl))),(C (Constant c))) -> raise (Not_unifiable)
            |((F (Function (f1,tl1))),(F (Function (f2,tl2)))) -> if f1 = f2 && leng tl1 = leng tl2 
                            then hold_fun mgu1 o tl1 tl2
                            else raise (Not_unifiable)
                            
let mgu t1 t2 = mgu1 t1 t2 (Substitution.empty)

let rec mgu_predicate1 (Predicate (p1,l1)) (Predicate (p2,l2)) o = 
    if p1 = p2 then 
        match (l1,l2) with
        |([],[]) -> o
        |((h1::t1),(h2::t2) ) -> mgu_predicate1 (Predicate (p1,t1)) (Predicate (p2,t2)) (mgu1 h1 h2 o)
        |(_,_) -> raise (Not_unifiable)
    else raise (Not_unifiable)

let mgu_predicate (Predicate (p1,l1)) (Predicate (p2,l2)) = 
                mgu_predicate1 (Predicate (p1,l1)) (Predicate (p2,l2)) (Substitution.empty)

exception Not_unifiable


val leng1 : 'a list -> int -> int = <fun>


val leng : 'a list -> int = <fun>


File "[36]", lines 12-14, characters 4-61:
12 | ....match (l1,l2) with
13 |     | ([],[]) -> acc
14 |     |( (h1::t1), (h2::t2) ) -> hold_fun f (f h1 h2 acc) t1 t2
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
((_::_, [])|([], _::_))


val hold_fun : ('a -> 'b -> 'c -> 'c) -> 'c -> 'a list -> 'b list -> 'c =
  <fun>


val composition :
  term Substitution.t -> term Substitution.t -> term Substitution.t = <fun>


val mgu1 : term -> term -> term Substitution.t -> term Substitution.t = <fun>


val mgu : term -> term -> term Substitution.t = <fun>


val mgu_predicate1 :
  predicate -> predicate -> term Substitution.t -> term Substitution.t =
  <fun>


val mgu_predicate : predicate -> predicate -> term Substitution.t = <fun>


In [37]:
(* 15 points *)
assert (string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; Y -> X}");
assert (string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; Y -> X}");
assert (mgu (var_t "Y") (var_t "Y") = Substitution.empty);
assert (mgu (const_t "0") (const_t "0") = Substitution.empty);
assert (mgu (const_t "0") (var_t "Y") = Substitution.singleton (var "Y") (const_t "0"));
assert (
  match mgu (const_t "0") (const_t "1") with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (
  match mgu (func_t "f" [const_t "0"]) (func_t "g" [const_t "1"]) with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "X") (var_t "Y") ||
        mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "Y") (var_t "X"))

- : unit = ()


In [38]:
(* 15 points *)
let t1 = F (Function("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")])) in 
let t2 = F (Function("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")])) in
let u = mgu t1 t2 in
assert (Substitution.cardinal u = 3);
assert (string_of_substitution u = "{; X -> a; Y -> a; Z -> a}");


- : unit = ()


## Problem 5

Implement the function:

```ocaml
query : ?limit:int -> clause list -> predicate list -> predicate list list 
```

where 

* the first optional argument is the `limit` i.e, maximum number of results to be returned. By default, the limit is set to 10 (see code below).
* the second argument is the `program` which is a list of clauses.
* the third argument is the `goal` which is a list of predicates.

The function returns a list of predicate lists (`results`). Each of these results is a instance of the original goal and is a logical consequence of the program. If the given goal is not a logical consequence of the program, then the result is an empty list. See tests cases for expected results.

For the rule and goal order, choose what Prolog does; choose the left-most subgoal for goal order and first rule in the order in which the rules appear in the program for the rule order. 

Hint: Implement a purely functional recursive solution. The backtracking and choice points naturally fall out of the implementation. The reference solution is 17 lines long. 

In [39]:
let rec substitute_in_predicate_list1 s l acc = 
    match l with 
    |[] -> acc 
    |h::t -> substitute_in_predicate_list1 s t (acc @ [substitute_in_predicate s h])
    
let substitute_in_predicate_list s l = substitute_in_predicate_list1 s l []

let rec fun5 limit program resolvant g acc result = 
    if List.length (!result) < limit then (match resolvant with 
        |[] ->  result := !result @ [g]  
        |hd::tl -> (let rec fun45 lim' pro' resol' g' acc' result' = 
                        (match pro' with 
                            |[] -> ()
                            |x::xs -> (match freshen(x) with 
                                            |Fact f -> (match (mgu_predicate f hd) with
                                                            |exception Not_unifiable -> fun45 lim' xs resol' g' acc' result'
                                                            |o -> let goal1 = substitute_in_predicate_list o g' in
                                                                let resnew = substitute_in_predicate_list o resol' in
                                                                fun5 limit program resnew goal1 acc' result';
                                                                fun45 lim' xs resol' g' acc' result' 
                                                        )
                                            |Rule  (head,body) -> (match (mgu_predicate head hd) with 
                                                                        |exception Not_unifiable -> fun45 lim' xs resol' g' acc' result'
                                                                        |o -> let res1 = body @ resol' in
                                                                           let goalnew = substitute_in_predicate_list o g' in
                                                                           let resnew = substitute_in_predicate_list o res1 in
                                                                           fun5 limit program resnew goalnew acc' result' ;
                                                                           fun45 lim' xs resol' g' acc' result'
                                                                    )
                                        )
                        )
                     in fun45 limit program tl g acc result )
    ) 
    
    and query1 l p g acc res = fun5 l p g g acc res  ; !res
    

let query ?(limit=10) program goal = let result = ref [] in query1 limit program goal [] result
    

val substitute_in_predicate_list1 :
  term Substitution.t -> predicate list -> predicate list -> predicate list =
  <fun>


val substitute_in_predicate_list :
  term Substitution.t -> predicate list -> predicate list = <fun>


val fun5 :
  int ->
  clause list ->
  predicate list -> predicate list -> 'a -> predicate list list ref -> unit =
  <fun>
val query1 :
  int ->
  clause list ->
  predicate list -> 'a -> predicate list list ref -> predicate list list =
  <fun>


val query :
  ?limit:int -> clause list -> predicate list -> predicate list list = <fun>


In [40]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [41]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "c"]] in
assert (query p g = [])

- : unit = ()


In [42]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [var_t "X"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);

let g = [pred "f" [var_t "X"; var_t "Y"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]])

- : unit = ()


In [43]:
(* 10 points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in
let g = [ancestor (const_t "rickard") (const_t "ned")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [44]:
(* 20 points *)
(* Tests backtracking *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (const_t "rickard") (const_t "robb")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [45]:
(* 15 points *)
(* Tests choice points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (var_t "X") (const_t "robb")] in
let g1,g2 = match query p g with [v1;v2] -> v1,v2 | _ -> failwith "error" in
assert (g1 = [ancestor (const_t "ned") (const_t "robb")]);
assert (g2 = [ancestor (const_t "rickard") (const_t "robb")])

- : unit = ()


In [46]:
(* 15 points *)
(* Tests choice points *)
let nil = const_t "nil" in
let cons h t = func_t "cons" [h;t] in
let append x y z = pred "append" [x;y;z] in
let c1 = fact @@ append nil (var_t "Q") (var_t "Q") in
let c2 = rule (append (cons (var_t "H") (var_t "P")) (var_t "Q") (cons (var_t "H") (var_t "R")))
              [append (var_t "P") (var_t "Q") (var_t "R")] in
let p = [c1;c2] in

let g = [append (var_t "X") (var_t "Y") (cons (const_t "1") (cons (const_t "2") (cons (const_t "3") nil)))] in

let g' = query p g in
assert (List.length g' = 4)

- : unit = ()
